In [1]:
from utils.processing import *
import numpy as np
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Dict, Tuple, Union
from dataclasses import dataclass, field

ModuleNotFoundError: No module named 'category_encoders.catboost'

In [86]:
with open("config/config.yaml", 'r') as ymlfile:
    config = yaml.safe_load(ymlfile)
    
LABEL_NAMES = config['LABEL_NAMES'][0]

In [87]:
dev_df, _ = load_file("data/development.csv", sep=",", dtype = {"id": str, "page_rank": int, "label": int})
dev_df_original = dev_df.copy()

  Development set: 79,997 samples, 7 features


In [88]:
dev_df

,Id,source,title,article,page_rank,timestamp,label
0,0,AllAfrica.com,OPEC Boosts Nigeria&#39;s Oil Revenue By .82m Bpd,THE Organisation of Petroleum Exporting Countr...,5,2004-09-16 22:39:53,5
1,1,Xinhua,Yearender: Mideast peace roadmap reaches dead-...,Looking back at the major events that took pla...,5,2004-12-17 19:01:14,0
2,2,Yahoo,Battleground Dispatches for Oct. 5 \\n (CQP...,CQPolitics.com - Here are today's Battleground...,5,2006-10-05 18:42:29,0
3,3,BBC,Air best to resuscitate newborns,Air rather than oxygen should be used to resus...,5,0000-00-00 00:00:00,0
4,4,Yahoo,High tech German train crash kills at least on...,"<p><a href=""://us.rd.yahoo.com/dailynewshttp/r...",5,2006-09-22 17:28:57,0
...,...,...,...,...,...,...,...
79992,79992,Yahoo,Italy's embattled Prodi faces vote of confiden...,"<p><a href=""http://us.rd.yahoo.com/dailynews/r...",5,2008-01-23 11:39:35,0
79993,79993,All-Baseball.com,"Ding Dong, the Deal is Dead","As yesterday began, there was widespread antic...",5,0000-00-00 00:00:00,4
79994,79994,Yahoo,Two bombs discovered in Sardinia after Berlusc...,AFP - Police discovered two bombs near the Sar...,5,0000-00-00 00:00:00,0
79995,79995,Voice,Red Cross Report Alleges US Detainee Abuse at ...,A report by the International Committee of the...,5,0000-00-00 00:00:00,3


In [89]:
dev_df = apply_domain_extraction(dev_df)
dev_df = apply_title_extraction(dev_df)


dic1 = top_feature_by_category(dev_df, 5, 'title_suffix', 0.95)
dic2 = top_feature_by_category(dev_df, 5, 'first_link_domain', 0.95)

In [90]:
dev_df

,Id,source,title,article,page_rank,timestamp,label,first_link_domain,title_suffix
0,0,AllAfrica.com,OPEC Boosts Nigeria&#39;s Oil Revenue By .82m Bpd,THE Organisation of Petroleum Exporting Countr...,5,2004-09-16 22:39:53,5,NaN,NaN
1,1,Xinhua,Yearender: Mideast peace roadmap reaches dead-...,Looking back at the major events that took pla...,5,2004-12-17 19:01:14,0,NaN,NaN
2,2,Yahoo,Battleground Dispatches for Oct. 5 \\n (CQP...,CQPolitics.com - Here are today's Battleground...,5,2006-10-05 18:42:29,0,NaN,NaN
3,3,BBC,Air best to resuscitate newborns,Air rather than oxygen should be used to resus...,5,0000-00-00 00:00:00,0,NaN,NaN
4,4,Yahoo,High tech German train crash kills at least on...,"<p><a href=""://us.rd.yahoo.com/dailynewshttp/r...",5,2006-09-22 17:28:57,0,news.yahoo.com,NaN
...,...,...,...,...,...,...,...,...,...
79992,79992,Yahoo,Italy's embattled Prodi faces vote of confiden...,"<p><a href=""http://us.rd.yahoo.com/dailynews/r...",5,2008-01-23 11:39:35,0,us.rd.yahoo.com,NaN
79993,79993,All-Baseball.com,"Ding Dong, the Deal is Dead","As yesterday began, there was widespread antic...",5,0000-00-00 00:00:00,4,NaN,NaN
79994,79994,Yahoo,Two bombs discovered in Sardinia after Berlusc...,AFP - Police discovered two bombs near the Sar...,5,0000-00-00 00:00:00,0,NaN,AFP
79995,79995,Voice,Red Cross Report Alleges US Detainee Abuse at ...,A report by the International Committee of the...,5,0000-00-00 00:00:00,3,NaN,NaN


In [91]:
temp = add_is_label_feature_columns(dev_df, feature = "title_suffix", dict_top_values=dic1)
temp = add_is_label_feature_columns(temp,  feature = "first_link_domain", dict_top_values=dic2)
temp = add_links_numerics(temp)


In [92]:
dic2

{0: ['topix.net'],
 1: ['feeds.fool.com'],
 2: ['ad.uk.doubleclick.net',
  'adlog.com.com',
  'blogs.cnet.com',
  'cnet.com',
  'feeds.computerworld.com'],
 5: ['rss.cnn.com', 'newsisfree.com']}

In [93]:
temp

,Id,source,title,article,page_rank,timestamp,label,first_link_domain,title_suffix,is_0_title_suffix,...,is_4_title_suffix,is_6_title_suffix,is_0_first_link_domain,is_1_first_link_domain,is_2_first_link_domain,is_5_first_link_domain,n_links,n_images,n_ads,n_feeds
0,0,AllAfrica.com,OPEC Boosts Nigeria&#39;s Oil Revenue By .82m Bpd,THE Organisation of Petroleum Exporting Countr...,5,2004-09-16 22:39:53,5,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Xinhua,Yearender: Mideast peace roadmap reaches dead-...,Looking back at the major events that took pla...,5,2004-12-17 19:01:14,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Yahoo,Battleground Dispatches for Oct. 5 \\n (CQP...,CQPolitics.com - Here are today's Battleground...,5,2006-10-05 18:42:29,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
3,3,BBC,Air best to resuscitate newborns,Air rather than oxygen should be used to resus...,5,0000-00-00 00:00:00,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Yahoo,High tech German train crash kills at least on...,"<p><a href=""://us.rd.yahoo.com/dailynewshttp/r...",5,2006-09-22 17:28:57,0,news.yahoo.com,NaN,0,...,0,0,0,0,0,0,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79992,79992,Yahoo,Italy's embattled Prodi faces vote of confiden...,"<p><a href=""http://us.rd.yahoo.com/dailynews/r...",5,2008-01-23 11:39:35,0,us.rd.yahoo.com,NaN,0,...,0,0,0,0,0,0,2,1,0,0
79993,79993,All-Baseball.com,"Ding Dong, the Deal is Dead","As yesterday began, there was widespread antic...",5,0000-00-00 00:00:00,4,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
79994,79994,Yahoo,Two bombs discovered in Sardinia after Berlusc...,AFP - Police discovered two bombs near the Sar...,5,0000-00-00 00:00:00,0,NaN,AFP,0,...,0,0,0,0,0,0,0,0,0,0
79995,79995,Voice,Red Cross Report Alleges US Detainee Abuse at ...,A report by the International Committee of the...,5,0000-00-00 00:00:00,3,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0


# Build pipeline

In [ ]:
from sklearn.model_selection import train_test_split


df_clean = preprocessing_data(dev_df_original)

X = df_clean.drop('label', axis=1)
y = df_clean['label']



X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

Converting timestamp to datetime...
Dropping duplicates based on source, title, article, label keeping the most recent one...
Dropping duplicates that have the same source, title, article but different label...
  Preprocessed Data: 77,180 samples, 7 features


In [11]:
transformer1 = TopFeatureTransformer(feature = "title_suffix")
transformer2 = TopFeatureTransformer(feature = "first_link_domain")

transformer1.fit(X_train, y_train)
transformer2.fit(X_train, y_train)

Fitting with top_n=5, threshold=0.95
Fitting with top_n=5, threshold=0.95


,top_n,5
,feature,'first_link_domain'
,threshold,0.95
,keep_column,True


In [12]:
X_train = transformer1.transform(X_train)
X_train = transformer2.transform(X_train)


X_val = transformer1.transform(X_val)
X_val = transformer2.transform(X_val)

In [13]:
X_val[X_val['title_suffix'] == 'InfoWorld']

,Id,source,title,article,page_rank,timestamp,title_suffix,is_0_title_suffix,is_2_title_suffix,is_3_title_suffix,is_6_title_suffix,first_link_domain,is_0_first_link_domain,is_1_first_link_domain,is_2_first_link_domain,is_5_first_link_domain
29385,29385,Yahoo,Microsoft ponders licensing for app virtualiza...,"InfoWorld - SANTA CLARA, CALIF. -- Noting two ...",5,NaT,InfoWorld,0,1,0,0,NaN,0,0,0,0
44222,44222,Yahoo,In Brief: IBM to upgrade Rational Software Arc...,InfoWorld - IBM this month plans to release an...,5,NaT,InfoWorld,0,1,0,0,NaN,0,0,0,0
55165,55165,Yahoo,IBM offers developers modeling transition tool...,InfoWorld - Looking to enable transitions betw...,5,NaT,InfoWorld,0,1,0,0,NaN,0,0,0,0


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import TargetEncoder

# Definisci quali colonne trasformare e quali lasciare invariate
preprocessor = ColumnTransformer(
    transformers=[
        ('source_enc', TargetEncoder(target_type='multiclass'), ['source'])
    ],
    remainder='passthrough', # <--- Fondamentale: mantiene le altre colonne!
    verbose_feature_names_out=False
).set_output(transform="pandas")

X_train_processed = preprocessor.fit_transform(X_train, y_train)

In [15]:
X_train_processed['source'] = X_train['source']

In [16]:
X_train_processed[X_train_processed['source'] == "Syfy.com"]


,source_0,source_1,source_2,source_3,source_4,source_5,source_6,Id,title,article,...,is_0_title_suffix,is_2_title_suffix,is_3_title_suffix,is_6_title_suffix,first_link_domain,is_0_first_link_domain,is_1_first_link_domain,is_2_first_link_domain,is_5_first_link_domain,source
57029,0.0,0.0,0.0,1.0,0.0,0.0,0.0,57029,Cage uses karaoke as his stress buster!,Hollywood star Nicolas Cage has revealed that ...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
24509,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24509,Shahid finally pays his tribute to Aamir,Shahid has been an unabashed fan of Aamir Khan...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
105,0.0,0.0,0.0,1.0,0.0,0.0,0.0,105,"Alka, Aadesh tiff cause problems for 'Voice of...",There is more to the current crisis on popular...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
38483,0.0,0.0,0.0,1.0,0.0,0.0,0.0,38483,Elton John plays English miner in cameo stage ...,British pop superstar Elton John made a brief ...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
31517,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31517,Actor Madhavan dons the director cap,Southern Superstar Madhavan is all set to don...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10799,Terry Pratchett says he has Alzheimer's,Best-selling fantasy author Terry Pratchett ha...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
28722,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28722,Suneel Darshan: Upen is Akki's replica,Has elder brother Suneel Darshan joined hands...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com
49638,0.0,0.0,0.0,1.0,0.0,0.0,0.0,49638,Gauri Khan turns model,"Ok, so we have seen Shahrukh Khan in countless...",...,0,0,0,0,NaN,0,0,0,0,Syfy.com
4036,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4036,Priyanka plays schoolgirl in KMG sequel,The sizzling Priyanka Chopra is all set to ton...,...,0,0,0,0,NaN,0,0,0,0,Syfy.com


In [20]:
!pip install category_encoders

Looking in indexes: https://****@pkgs.dev.azure.com/NPIDemandPlanning/_packaging/npi-pack-feed/pypi/simple/, https://pypi.org/simple/


In [21]:
from category_encoders.cat_boost import CatBoostEncoder

In [23]:
X_train

,Id,source,title,article,page_rank,timestamp
33563,33563,Ottawa,Smitherman must go for kids&#39; sake,For the Ottawa Sun. If you believe in the futu...,5,2004-10-21 10:20:35
66324,66324,Atlanta,Kerry lambastes Bush over &#39;failures&#39; i...,WASHINGTON - The Iraq war moved front and cent...,5,NaT
67926,67926,Yahoo,British men are better drivers than women: top...,AFP - The man in charge of driving test standa...,5,NaT
69529,69529,Register,<strong>[NSFW]</strong> eBay in gearbox sex do...,<h4>'Modded at rear' but box unaffected</h4>\\...,5,2007-02-07 18:01:50
43501,43501,Yahoo,&quot;SNL&quot; musical performances due on Ta...,Reuters - The first &quot;Saturday Night Live&...,5,2007-09-06 01:51:27
...,...,...,...,...,...,...
24223,24223,Poughkeepsie,Santana&#39;s pitching sends Minnesota to Game...,We have seen this before from the New York Yan...,5,2004-10-06 15:00:03
63143,63143,CNET,"Plaxo gets naked, opens up some source code",A small step for the budding social network's ...,3,2007-08-29 08:51:09
54272,54272,Yahoo,Disney star appears nude in Web photo \\n (...,"<p><a href=""http://us.rd.yahoo.com/dailynews/r...",5,2007-09-08 00:00:36
26983,26983,New,S-q-u-a-b-b-l-e,Third-party applications are supposedly the se...,5,2008-01-20 08:31:29


In [33]:
CatBoostEnc = CatBoostEncoder(cols=['source'], return_df=True, verbose = 1, sigma = 1e-5)

CatBoostEnc.fit(X_train, y_train)

,verbose,1
,cols,['source']
,drop_invariant,False
,return_df,True
,handle_unknown,'value'
,handle_missing,'value'
,random_state,None
,sigma,1e-05
,a,1


In [39]:
for i in range(7): # Per le tue 7 label
    target_binary = (y_train == i).astype(int)
    enc = CatBoostEncoder(cols=['source'], return_df=True, verbose = 1, sigma = 1e-5)
    X_train[f'source_prob_class_{i}'] = enc.fit_transform(X_train['source'], target_binary)
    X_val[f'source_prob_class_{i}'] = enc.transform(X_val['source'])



In [81]:
from sklearn.model_selection import train_test_split


df_clean = preprocessing_data(dev_df_original)
df_clean = df_clean.sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = apply_domain_extraction(df_clean)
df_clean = apply_title_extraction(df_clean)

X = df_clean.drop('label', axis=1)
y = df_clean['label']



X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

Converting timestamp to datetime...
Dropping duplicates based on source, title, article, label keeping the most recent one...
Dropping duplicates that have the same source, title, article but different label...
  Preprocessed Data: 77,180 samples, 7 features


In [82]:
text_encder = TextEncoder(cols=('source', 'first_link_domain','title_suffix'), sigma=1e-5)

X_train_new = text_encder.fit_transform(X_train, y_train)

In [83]:
X_train_new

,Id,title,article,page_rank,timestamp,source_prob_cl_0,first_link_domain_prob_cl_0,title_suffix_prob_cl_0,source_prob_cl_1,first_link_domain_prob_cl_1,...,title_suffix_prob_cl_3,source_prob_cl_4,first_link_domain_prob_cl_4,title_suffix_prob_cl_4,source_prob_cl_5,first_link_domain_prob_cl_5,title_suffix_prob_cl_5,source_prob_cl_6,first_link_domain_prob_cl_6,title_suffix_prob_cl_6
31982,71704,Prescription drug ads simply not necessary,A major component in the skyrocketing cost of ...,5,NaT,0.294784,0.294779,0.294779,0.133341,0.133342,...,0.123688,0.108204,0.108206,0.108205,0.161958,0.161958,0.161962,0.038158,0.038158,0.038157
2185,20207,Art Review: Spelling It Out in Black and White,Black and white portraiture dominates âIslan...,5,2007-10-21 12:18:00,0.294783,0.147392,0.147391,0.133342,0.066671,...,0.061844,0.108206,0.054102,0.054102,0.161957,0.080979,0.080980,0.038157,0.519076,0.519071
52715,41077,Cycling: Tour stage nine,Juan Mauricio Soler Hernandez comes home befor...,5,2007-07-17 17:37:37,0.294784,0.098261,0.098259,0.133341,0.044447,...,0.374562,0.108204,0.036068,0.036069,0.161959,0.053986,0.053987,0.038157,0.346055,0.346053
47652,16795,Nintendo Operating Profit Up 39 Percent,TOKYO (Reuters) - Japanese video game maker N...,5,NaT,0.294782,0.323692,0.323696,0.133342,0.033335,...,0.280923,0.108204,0.027052,0.027051,0.161958,0.040490,0.040490,0.038158,0.259540,0.259541
59185,2547,Russia agrees to US deal that writes off $33bn...,RUSSIA finally signed up yesterday to an agree...,5,NaT,0.294781,0.258957,0.258960,0.133340,0.026668,...,0.224740,0.108205,0.021641,0.021641,0.161961,0.032392,0.032392,0.038158,0.207633,0.207631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44968,46547,Steelmaker Corus has first profit,High steel prices help Anglo-Dutch steelmaker ...,5,NaT,0.547200,0.282312,0.275641,0.062612,0.135696,...,0.123723,0.009398,0.113147,0.106498,0.278694,0.167058,0.174434,0.026369,0.042596,0.038020
9652,64983,Microsoft adds IP indemnity to Linux fight,"Borrowing a line from Allstate Insurance, Micr...",5,2004-11-10 13:58:49,0.042112,0.282316,0.275642,0.019049,0.135692,...,0.123719,0.015458,0.113144,0.106493,0.165996,0.167070,0.174449,0.005451,0.042594,0.038019
63819,74707,David Sadler column: Festive title hopes,SIR Alex Ferguson believes he will have a clea...,5,NaT,0.080924,0.282303,0.275637,0.008334,0.135691,...,0.123717,0.319260,0.113141,0.106494,0.260122,0.167069,0.174447,0.002385,0.042594,0.038019
28129,77462,Blue Moon over North America Tonight,"At 9:04 pm Eastern Daylight Time on May 31st, ...",3,2007-06-01 00:10:27,0.195942,0.282302,0.275629,0.000064,0.135688,...,0.123735,0.000052,0.113139,0.106493,0.000077,0.167063,0.174438,0.000018,0.042593,0.038018


In [84]:
X_train

,Id,source,title,article,page_rank,timestamp,first_link_domain,title_suffix
31982,71704,Jackson,Prescription drug ads simply not necessary,A major component in the skyrocketing cost of ...,5,NaT,NaN,NaN
2185,20207,New,Art Review: Spelling It Out in Black and White,Black and white portraiture dominates âIslan...,5,2007-10-21 12:18:00,NaN,NaN
52715,41077,BBC,Cycling: Tour stage nine,Juan Mauricio Soler Hernandez comes home befor...,5,2007-07-17 17:37:37,NaN,NaN
47652,16795,Reuters,Nintendo Operating Profit Up 39 Percent,TOKYO (Reuters) - Japanese video game maker N...,5,NaT,NaN,NaN
59185,2547,Times,Russia agrees to US deal that writes off $33bn...,RUSSIA finally signed up yesterday to an agree...,5,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...
44968,46547,BBC,Steelmaker Corus has first profit,High steel prices help Anglo-Dutch steelmaker ...,5,NaT,NaN,NaN
9652,64983,ZDNet.co.uk,Microsoft adds IP indemnity to Linux fight,"Borrowing a line from Allstate Insurance, Micr...",5,2004-11-10 13:58:49,NaN,NaN
63819,74707,Manchester,David Sadler column: Festive title hopes,SIR Alex Ferguson believes he will have a clea...,5,NaT,NaN,NaN
28129,77462,RedNova,Blue Moon over North America Tonight,"At 9:04 pm Eastern Daylight Time on May 31st, ...",3,2007-06-01 00:10:27,NaN,NaN
